<a href="https://colab.research.google.com/github/portoforigin/deeplearn/blob/main/FastAI_PETS_Cat_Dog_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Install Dependencies:**

 - Default version of fastai in Google Colab is 1.0.61(?), we need >v2.2.5

In [ ]:
!pip install -Uqq fastai
!pip install lime

import fastai
fastai.__version__

PROJECT_NAME = "FastAI_PETS"
MODEL_NAME = "fastai_pets_cat_dog_001.pkl"

**Connect to Google Drive**

In [ ]:
from fastai.vision.all import *
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'deeplearn/'
dest=Path(base_dir + PROJECT_NAME)
try:
  dest.mkdir(parents=True, exist_ok=False)
except FileExistsError:
  print ('File Already Exists')

## **PETS Dataset**

Load a view the PETS dataset:

In [ ]:
from fastai.vision.all import *

path = untar_data(URLs.PETS)
# path.ls()

files = get_image_files(path/"images")
print("There are %s images" % (len(files)))

# Label Function
# - Capital first letter indicates Cat
def label_func(f): return f[0].isupper()

# Load Data set
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=label_func, item_tfms=Resize(224),
    metrics=error_rate)

**Load pre-trained Model**
Note: All custom functions need to be defined before calling load_leaner() (i.e. `def label_func`)

In [ ]:
print(f"Loading Model from {dest/MODEL_NAME}")
learn = load_learner(dest/MODEL_NAME)


In [ ]:
# View Data set (True == Cat, False == Dog)
dls.show_batch()

## **Generate Fast AI Model for identifying Cats vs. Dogs**

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

## **Evaulate Model**

In [ ]:
#Print top 20 losses. 
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
num_losses = 5
losses, indices = interp.top_losses(num_losses)
x = 1
for idx in indices:
    print(f"[{x}] {dls.valid_ds.items[idx]}")
    x += 1
interp.plot_top_losses(num_losses)

## **Upload Test Image**

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

for fname in uploaded.keys():
  is_cat,_,probs = learn_species.predict(fname)

  # Show Image
  img = cv2.imread(fname)
  img_cvt=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.imshow(img_cvt)
  plt.show()

  print(f"Is %s a cat?: %s" % (fname, is_cat))
  print(f"Probability it's a cat: {probs[1].item():.6f}")

## **Save Model to Drive**

In [ ]:
learn.export(dest/MODEL_NAME)